# GameDay2020 Notebook

The standard dilemma of fantasy baseball is who to choose next: the best available player but in a position you've already filled?; the best at a rare position?; the next best closer because they're going fast?  Whatever choice you make will have consequences, as the next player will no longer have your pick as a choice, and so on.  Ideally you would forecast how each of those would play out and take the best option --- that's exactly what this code does: it identifies the best player left in each of the positions you still need to fill, simulates the rest of draft in each of those scenarios, and returns the best choice.

And it's easy to use!  After [cloning](https://github.com/wrapgenius/GameDay2020) GameDay2020 (and provided you have Jupyter Notebooks with Python 3 installed, and a spreadsheet program) you should have everything you need to do a live roto draft.

The code is based on three ingredients: *projections*, *rankings*, and *injuries*:  
- Projections are estimates of player performances for 2020, and are based on data scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) (model = ZiPS, Steamer, or TheBat.) 
- Rankings are draft-order recommendations.  The default (i.e.; ranking_method = 'Yahoo') is to use Yahoo Rankings (from Feb 22, 2020).  Others include [ESPN](https://www.espn.com/fantasy/baseball/story/_/id/28285423/fantasy-baseball-2020-rankings-roto-rotisserie), [FantasyPros](https://www.fantasypros.com/mlb/rankings/overall.php), and [Rotographs](https://fantasy.fangraphs.com), or you can put in your own.  Yahoo for yahoo leagues makes most sense since it most closely imitates autodraft.    
- Injured players to be removed from draft consideration are located in the Injuries2020.xlsx file found in the Injured_List_Spreadsheets directory. 

_Your_ roto league may have more or less than 12 teams, or use different stats; that ok!  Declare them when defining the Draft object: most of the obvious stats are included, more obscure ones may require hacking into the [fangraphs_projection_2020](https://github.com/wrapgenius/GameDay2020/blob/master/GameDayFunctions/fangraphs_projection_2020.py).  

Note, if you choose to shuffle_picks = True, it will not default to the next best pick, but draw a random number between 0 and 1 and compare it to the sigmoid function 1/1+exp(-(pick_number - avg_pick_number)).  So say it's the 15th pick and Trevor Story, who is typically drafted 10th is available, there is an 85% chance you'll select him.  Also note, it can't _yet_ do AL or NL only. 

Below are examples of the different ways the code can be run.  

In [1]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = False

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = 'Yahoo')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer',ranking_method = 'FantasyPros')
player_ranking_thebat = Projection(path_data=path_data,year=year,model='TheBat',ranking_method = 'ESPN')
player_rankings = {"ZiPS":player_ranking_zips, "Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}

Next need a **Draft** object.  The default, e.g.; Draft(player_rankings['ZiPS']), uses 12 teams and the roster spots/roto stats shown in this example. 

In [4]:
# Enter your roto-league-specific stats.  The default settings are shown here.  
draft_position = 2
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 

# Choose the projection system you prefer ()
player_projections = player_rankings['ZiPS']

# Get an instance of the Draft Class with your league-specific details and projection preference.
zips_example = Draft(player_projections, 
                     draft_position = draft_position, 
                     number_teams = number_teams,
                     roster_spots = roster_spots,
                     batter_stats = batter_stats,
                     pitcher_stats = pitcher_stats)

To use while drafting in real-time, simply record the draft results in a spreadsheet (.xlsx, so if you're using Numbers, need to export to Excel) and run the cell. 

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [5]:
# Example pick in the 2nd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround2.xlsx', path_list = path_drafts)

Finding Best Pick For Team 5
Stored Result for Pick 0 [20/12] J.D. Martinez OF whose placement/score is 7/137.5
Stored Result for Pick 1 [20/21] Xander Bogaerts SS whose placement/score is 8/125.0
Stored Result for Pick 2 [20/22] Jack Flaherty SP whose placement/score is 2/160.5
Stored Result for Pick 3 [20/23] Walker Buehler P whose placement/score is 1/161.5
Stored Result for Pick 4 [20/24] Ketel Marte 2B whose placement/score is 5/138.0
Stored Result for Pick 5 [20/25] Rafael Devers 3B whose placement/score is 9/113.5
Stored Result for Pick 6 [20/26] Trea Turner UTIL whose placement/score is 9/116.5
Stored Result for Pick 7 [20/27] Roberto Osuna RP whose placement/score is 1/161.5
Stored Result for Pick 8 [20/45] DJ LeMahieu 1B whose placement/score is 3/161.5
Not Storing Result for Pick 9 [20/185] J.T. Realmuto C
Best Pick is Walker Buehler putting you in 5 place


In [6]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [7]:
# The projected_stats give the raw numbers

projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,781.0,830.0,271.0,32.0,219.0,739.0,168.0,436.0,0.275173,0.814101,50.0,33.0,3.0,3.0,27.0,246.0,921.0,3.577590,1.180056,2.0
1,637.0,766.0,240.0,34.0,139.0,566.0,101.0,388.0,0.272891,0.775723,48.0,29.0,2.0,2.0,0.0,224.0,790.0,3.506026,1.179437,0.0
2,688.0,745.0,256.0,29.0,180.0,632.0,75.0,452.0,0.276000,0.819536,54.0,30.0,2.0,2.0,106.0,223.0,964.0,3.306995,1.086878,8.0
3,715.0,789.0,263.0,23.0,222.0,765.0,52.0,538.0,0.273162,0.827543,44.0,34.0,1.0,1.0,71.0,234.0,810.0,3.742806,1.192475,5.0
4,607.0,601.0,215.0,23.0,207.0,691.0,55.0,460.0,0.270574,0.849310,63.0,35.0,3.0,3.0,47.0,257.0,1102.0,3.383327,1.118416,4.0
5,484.0,536.0,200.0,20.0,121.0,477.0,45.0,325.0,0.282496,0.829994,42.0,29.0,2.0,2.0,46.0,262.0,803.0,3.551960,1.220004,4.0
6,636.0,761.0,240.0,28.0,159.0,575.0,107.0,435.0,0.281997,0.817889,51.0,34.0,3.0,3.0,55.0,240.0,876.0,3.625623,1.189397,5.0
7,779.0,750.0,239.0,23.0,256.0,809.0,76.0,588.0,0.268408,0.843174,67.0,40.0,4.0,4.0,23.0,279.0,1064.0,3.499018,1.149155,2.0
8,713.0,681.0,248.0,25.0,250.0,787.0,85.0,501.0,0.270014,0.850782,41.0,29.0,1.0,1.0,0.0,187.0,671.0,3.555951,1.186404,0.0
9,540.0,507.0,175.0,20.0,173.0,544.0,43.0,422.0,0.263956,0.839222,62.0,42.0,5.0,5.0,20.0,234.0,922.0,3.681249,1.175104,2.0


In [8]:
# And projected_scores show how they translate to roto scores

projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,12.0,12.0,12.0,10.5,7.0,8.5,12.0,5.0,8.0,3.0,5.5,7.5,9.0,9.0,5.0,4.0,8.0,5.0,6.0,9.0
1,5.0,9.0,5.5,12.0,2.0,3.0,9.0,2.0,6.0,1.0,4.0,11.0,5.5,5.5,1.5,10.0,3.0,9.0,7.0,11.5
2,6.0,5.0,9.5,9.0,5.0,5.0,5.0,7.0,9.0,5.0,8.5,9.0,5.5,5.5,12.0,11.0,10.0,12.0,12.0,1.5
3,9.0,10.0,11.0,4.5,8.5,10.0,3.0,11.0,7.0,6.0,3.0,5.5,2.0,2.0,10.0,6.5,5.0,1.0,3.0,3.5
4,3.0,3.0,3.0,4.5,6.0,6.0,4.0,8.0,5.0,11.0,11.0,4.0,9.0,9.0,7.0,3.0,12.0,11.0,11.0,6.0
5,1.0,2.0,2.0,1.5,1.0,1.0,2.0,1.0,12.0,7.0,2.0,11.0,5.5,5.5,6.0,2.0,4.0,7.0,1.0,6.0
6,4.0,8.0,5.5,8.0,3.0,4.0,11.0,4.0,11.0,4.0,7.0,5.5,9.0,9.0,9.0,5.0,6.0,4.0,4.0,3.5
7,11.0,6.0,4.0,4.5,12.0,12.0,6.0,12.0,3.0,10.0,12.0,2.0,11.0,11.0,4.0,1.0,11.0,10.0,9.0,9.0
8,8.0,4.0,7.0,7.0,11.0,11.0,8.0,10.0,4.0,12.0,1.0,11.0,2.0,2.0,1.5,12.0,1.0,6.0,5.0,11.5
9,2.0,1.0,1.0,1.5,4.0,2.0,1.0,3.0,1.0,9.0,10.0,1.0,12.0,12.0,3.0,6.5,9.0,3.0,8.0,9.0


In [9]:
# Finally, the projected_standings shows the resulting standings

projected_standings

7     160.5
0     158.0
2     152.5
10    142.0
4     136.5
8     135.0
11    127.5
6     124.5
1     122.5
3     121.5
9      99.0
5      80.5
dtype: float64

In [12]:
# But if you just want to know how your team finishes, that's projected_place

projected_place

5

### You can simulate drafts as well. Naive drafts simply picks the next eligible (i.e., open roster spot) player on the ranked list. 

In [13]:
# Try a quick draft: 4 teams, limited roster spots, naive (i.e., next available pick on ranked list) picks.

naive_draft = True
zipsS= Draft(player_rankings['ZiPS'],draft_position = 2,number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})
zipsS.draft_all(naive_draft = naive_draft, shuffle_picks = shuffle_picks)
print('Placement = '+str(zipsS.roto_placement))

Placement = 3


In [14]:
# Now try a naive draft again, but for full 12 team and all 24 roster spots.  Draft Position is 2 

naive_draft = True
zipsNaive_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zipsNaive_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_2.roto_placement))

Placement  = 3


In [10]:
# This time do the full-simulation, recommended picks version, i.e.; naive_draft = False
# Remember, it will try every remaining open position and pick the one whose resulting placement is highest.  
# Setting silent = False shows the decision logic for each pick (highest placement followed by highest score [placement, score])

naive_draft = False
zips_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zips_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks,silent=False)
print('Placement  = '+str(zips_2.roto_placement))

Team 1 Drafting Christian Yelich for OF
Stored Result for Pick 0 [2/2] Mookie Betts OF whose placement/score is 9/120.5
Stored Result for Pick 1 [2/3] Mike Trout UTIL whose placement/score is 8/128.5
Stored Result for Pick 2 [2/6] Anthony Rendon 3B whose placement/score is 11/110.0
Stored Result for Pick 3 [2/7] Cody Bellinger 1B whose placement/score is 10/118.0
Stored Result for Pick 4 [2/9] Alex Bregman SS whose placement/score is 7/130.0
Stored Result for Pick 5 [2/11] Justin Verlander SP whose placement/score is 12/107.5
Stored Result for Pick 6 [2/13] Jacob deGrom P whose placement/score is 9/113.5
Stored Result for Pick 7 [2/15] Whit Merrifield 2B whose placement/score is 9/113.0
Stored Result for Pick 8 [2/27] Roberto Osuna RP whose placement/score is 12/101.0
Not Storing Result for Pick 9 [2/185] J.T. Realmuto C
Team 2 picking Alex Bregman for SS
Team 3 Drafting Mookie Betts for OF
Team 4 Drafting Mike Trout for OF
Team 5 Drafting Ronald Acuna for OF
Team 6 Drafting Juan Soto 

In [17]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = zips_2.roto_team_stats
projected_standings = zips_2.roto_standings
projected_place = zips_2.roto_placement
projected_scores = zips_2.roto_team_stats_rank

In [18]:
projected_place

1

In [20]:
projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,742.0,742.0,266.0,21.0,258.0,862.0,107.0,487.0,0.280844,0.871629,49.0,32.0,2.0,2.0,53.0,231.0,801.0,3.542968,1.172234,4.0
1,766.0,837.0,276.0,27.0,241.0,755.0,90.0,485.0,0.278543,0.835851,59.0,42.0,4.0,4.0,20.0,253.0,962.0,3.755108,1.176550,2.0
2,723.0,834.0,271.0,29.0,182.0,674.0,75.0,501.0,0.273818,0.801056,46.0,32.0,2.0,2.0,56.0,236.0,792.0,3.614620,1.208681,4.0
3,650.0,657.0,225.0,36.0,188.0,640.0,138.0,423.0,0.269274,0.820786,39.0,27.0,0.0,0.0,56.0,215.0,654.0,3.770348,1.248440,5.0
4,644.0,610.0,234.0,19.0,203.0,638.0,71.0,469.0,0.262653,0.823494,60.0,40.0,3.0,3.0,47.0,241.0,975.0,3.605200,1.139460,3.0
5,700.0,729.0,254.0,38.0,199.0,693.0,87.0,509.0,0.270411,0.823188,74.0,36.0,4.0,4.0,65.0,258.0,1268.0,3.103707,1.041448,5.0
6,714.0,811.0,273.0,26.0,208.0,768.0,69.0,528.0,0.273050,0.818020,40.0,28.0,2.0,2.0,46.0,180.0,630.0,3.795678,1.223037,4.0
7,673.0,729.0,227.0,21.0,205.0,697.0,68.0,367.0,0.284703,0.843354,46.0,33.0,2.0,2.0,29.0,249.0,764.0,3.772381,1.243655,2.0
8,493.0,559.0,176.0,20.0,148.0,523.0,66.0,327.0,0.276791,0.824721,57.0,32.0,3.0,3.0,70.0,234.0,983.0,3.340015,1.115836,6.0
9,719.0,778.0,251.0,35.0,203.0,695.0,79.0,467.0,0.276118,0.825265,55.0,34.0,3.0,3.0,0.0,243.0,1012.0,3.364249,1.123387,0.0


In [21]:
projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,11.0,8.0,9.0,4.5,12.0,12.0,11.0,8.0,11.0,12.0,6.0,8.0,4.5,4.5,8.0,10.0,6.0,8.0,8.0,5.5
1,12.0,12.0,12.0,7.0,11.0,10.0,9.0,7.0,10.0,10.0,10.0,1.0,11.5,11.5,2.0,2.0,8.0,4.0,7.0,10.5
2,10.0,11.0,10.0,8.0,3.0,6.0,6.0,10.0,7.0,1.0,3.5,8.0,4.5,4.5,9.5,7.0,4.0,6.0,4.0,5.5
3,4.0,3.0,3.0,11.0,4.0,4.0,12.0,3.0,3.0,5.0,1.0,12.0,1.0,1.0,9.5,11.0,2.0,3.0,1.0,2.5
4,3.0,2.0,6.0,1.0,7.5,3.0,5.0,6.0,1.0,7.0,11.0,2.0,9.0,9.0,6.5,6.0,9.0,7.0,9.0,8.5
5,7.0,6.5,8.0,12.0,6.0,7.0,8.0,11.0,4.0,6.0,12.0,4.0,11.5,11.5,11.0,1.0,12.0,12.0,12.0,2.5
6,8.0,10.0,11.0,6.0,10.0,11.0,4.0,12.0,6.0,3.0,2.0,11.0,4.5,4.5,5.0,12.0,1.0,1.0,3.0,5.5
7,6.0,6.5,4.0,4.5,9.0,9.0,3.0,2.0,12.0,11.0,3.5,6.0,4.5,4.5,3.0,4.0,3.0,2.0,2.0,10.5
8,1.0,1.0,1.0,2.5,1.0,1.0,2.0,1.0,9.0,8.0,9.0,8.0,9.0,9.0,12.0,8.0,10.0,11.0,11.0,1.0
9,9.0,9.0,7.0,10.0,7.5,8.0,7.0,5.0,8.0,9.0,8.0,5.0,9.0,9.0,1.0,5.0,11.0,10.0,10.0,12.0


In [23]:
projected_standings

1     167.5
0     167.0
5     165.0
9     159.5
6     130.5
2     128.5
4     118.5
11    116.0
8     115.5
7     110.0
3      96.0
10     86.0
dtype: float64

In [16]:
# Lets compare Naive True/False for the 7th pick

naive_draft = True
zipsNaive_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zipsNaive_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_7.roto_placement))

Placement  = 10


In [17]:
naive_draft = False
zips_7 = Draft(player_rankings['TheBat'], draft_position = 2)
zips_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_7.roto_placement))

Team 2 picking Gerrit Cole for SP
Team 2 picking J.T. Realmuto for C
Team 2 picking Charlie Blackmon for OF
Team 2 picking Ozzie Albies for 2B
Team 2 picking Austin Meadows for OF
Team 2 picking Carlos Correa for SS
Team 2 picking Noah Syndergaard for SP
Team 2 picking Aroldis Chapman for RP
Team 2 picking Corey Seager for UTIL
Team 2 picking Raisel Iglesias for OF
Team 2 picking Mike Moustakas for 3B
Team 2 picking Brandon Lowe for BN
Team 2 picking Zac Gallen for P
Team 2 picking Sean Doolittle for P
Team 2 picking Sean Manaea for P
Team 2 picking Yonny Chirinos for RP
Team 2 picking Joc Pederson for 1B
Placement  = 5


In [18]:
naive_draft = False
zips_7 = Draft(player_rankings['TheBat'], draft_position = 2)
zips_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_7.roto_placement))

Team 2 picking Gerrit Cole for SP
Team 2 picking J.T. Realmuto for C
Team 2 picking Charlie Blackmon for OF
Team 2 picking Ozzie Albies for 2B
Team 2 picking Austin Meadows for OF
Team 2 picking Carlos Correa for SS
Team 2 picking Noah Syndergaard for SP
Team 2 picking Aroldis Chapman for RP
Team 2 picking Corey Seager for UTIL
Team 2 picking Raisel Iglesias for OF
Team 2 picking Mike Moustakas for 3B
Team 2 picking Brandon Lowe for BN
Team 2 picking Zac Gallen for P
Team 2 picking Sean Doolittle for P
Team 2 picking Sean Manaea for P
Team 2 picking Yonny Chirinos for RP
Team 2 picking Joc Pederson for 1B
Placement  = 5


In [19]:
# And for the 11th pick

naive_draft = True
zipsNaive_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zipsNaive_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_11.roto_placement))

Placement  = 8


In [20]:
naive_draft = False
zips_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zips_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_11.roto_placement))

Team 11 picking Whit Merrifield for 2B
Team 11 picking Jacob deGrom for SP
Team 11 picking Austin Meadows for OF
Team 11 picking Keston Hiura for UTIL
Team 11 picking Nick Castellanos for OF
Team 11 picking Sean Doolittle for RP
Team 11 picking Adam Ottavino for RP
Team 11 picking Andrew Benintendi for OF
Team 11 picking Tommy Edman for 3B
Team 11 picking Luis Severino for SP
Team 11 picking Yu Darvish for P
Team 11 picking J.T. Realmuto for C
Team 11 picking Sergio Romo for P
Team 11 picking Dustin May for BN
Team 11 picking Eric Hosmer for SS
Team 11 picking Christian Walker for 1B
Team 11 picking Charlie Morton for P
Placement  = 7


In [21]:
# We can do simlated drafts for different projection systems.  Above was all with ZiPs.  Lets try a Steamer, with custom league settings.

number_teams = 6
naive_draft  = True
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Placement  = 2


In [22]:
# And compare Naive to the recommendations from the simulation

naive_draft = False
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Team 2 picking Mookie Betts for OF
Team 2 picking DJ LeMahieu for 1B
Team 2 picking Francisco Lindor for SS
Team 2 picking Jack Flaherty for SP
Swapping UTIL with OF
Team 2 picking Yordan Alvarez for OF
Team 2 picking Bryce Harper for OF
Team 2 picking Chris Sale for SP
Team 2 picking Yoan Moncada for 3B
Team 2 picking Keone Kela for RP
Team 2 picking Jorge Polanco for BN
Team 2 picking Javier Baez for UTIL
Team 2 picking Zack Greinke for P
Team 2 picking Eloy Jimenez for BN
Team 2 picking Adam Ottavino for P
Team 2 picking Liam Hendriks for RP
Team 2 picking Max Muncy for 2B
Team 2 picking Ian Kennedy for P
Team 2 picking Gary Sanchez for C
Placement  = 1


In [23]:
# And lets compare Naive True/False for different draft positions

naive_draft = True
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Placement  = 12


In [24]:
naive_draft = False
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Team 2 picking Gerrit Cole for SP
Team 2 picking Gleyber Torres for OF
Team 2 picking Starling Marte for OF
Team 2 picking Patrick Corbin for 3B
Team 2 picking Paul Goldschmidt for 1B
Team 2 picking Jose Abreu for OF
Team 2 picking Eugenio Suarez for UTIL
Team 2 picking Jeff McNeil for 2B
Team 2 picking Corey Seager for SS
Team 2 picking Jesus Luzardo for SP
Team 2 picking Ken Giles for RP
Team 2 picking Salvador Perez for C
Team 2 picking Lorenzo Cain for BN
Team 2 picking Joe Jimenez for RP
Team 2 picking Sean Doolittle for P
Team 2 picking Griffin Canning for P
Team 2 picking Joe Musgrove for P
Placement  = 6


In [25]:
naive_draft = True
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Placement  = 10


In [26]:
naive_draft = False
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Team 7 picking Freddie Freeman for 1B
Team 7 picking J.D. Martinez for OF
Team 7 picking Shane Bieber for SP
Team 7 picking J.T. Realmuto for C
Team 7 picking Yu Darvish for SP
Team 7 picking Bo Bichette for SS
Team 7 picking Joey Gallo for OF
Team 7 picking Jeff McNeil for 3B
Swapping UTIL with OF
Team 7 picking Michael Conforto for OF
Team 7 picking Eduardo Escobar for 2B
Team 7 picking Trey Mancini for UTIL
Team 7 picking Miguel Sano for BN
Team 7 picking Kenta Maeda for RP
Team 7 picking Lance McCullers Jr. for P
Team 7 picking Carlos Martinez for RP
Team 7 picking Will Smith for P
Team 7 picking Joe Musgrove for P
Placement  = 1


In [27]:
naive_draft = True
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Placement  = 2


In [28]:
naive_draft = False
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Team 11 picking Justin Verlander for OF
Team 11 picking Freddie Freeman for 1B
Team 11 picking Ozzie Albies for 2B
Team 11 picking J.T. Realmuto for C
Team 11 picking Mike Clevinger for 3B
Team 11 picking Matt Olson for BN
Team 11 picking Gary Sanchez for UTIL
Team 11 picking Carlos Correa for SS
Team 11 picking Liam Hendriks for RP
Team 11 picking Dinelson Lamet for SP
Team 11 picking Lorenzo Cain for SP
Team 11 picking Kyle Schwarber for OF
Team 11 picking Sean Manaea for P
Team 11 picking Andrew Heaney for P
Team 11 picking Mallex Smith for OF
Team 11 picking Ian Kennedy for RP
Team 11 picking Marcus Stroman for P
Placement  = 7


In [29]:
# And finally, 7th pick using TheBat projection system. 

naive_draft = True
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Placement  = 2


In [30]:
naive_draft = False
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Swapping UTIL with 3B
Team 7 picking Nolan Arenado for 3B
Team 7 picking Jose Ramirez for UTIL
Team 7 picking Shane Bieber for SP
Team 7 picking Chris Sale for SP
Team 7 picking Eugenio Suarez for BN
Team 7 picking Zack Greinke for P
Team 7 picking Marcell Ozuna for OF
Team 7 picking Carlos Santana for 1B
Team 7 picking Mike Soroka for P
Team 7 picking Willson Contreras for C
Team 7 picking Mike Moustakas for 2B
Team 7 picking Scott Kingery for OF
Team 7 picking Tim Anderson for SS
Team 7 picking Justin Upton for OF
Team 7 picking Mark Melancon for RP
Team 7 picking Jesus Luzardo for RP
Team 7 picking Mike Foltynewicz for P
Placement  = 1
